In [55]:
# Import packages
from ultralytics import YOLO
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import warnings
warnings.filterwarnings("ignore")

## Using `haarcascade_frontalface_default` pre-trained model

In [56]:
def blur_face(img_path, blured_img_name = 'cascade_output.jpg', cascade_classifier = ".\pretrained_models\haarcascade_frontalface_default.xml", 
              scale_factor=1.1, min_neighbors=3, min_size=(30,30), blur_kernel_size=(25,25), blur_level=100, return_bb = False):

    """
    Blurs faces in the input image using the specified pre-trained Haar cascade classifier.

    Args:
    - img_path: string. Path to the input image.
    - blured_img_name: string. Name of the output blurred image file (default: 'output.jpg').
    - cascade_classifier: string. Path to the pre-trained Haar cascade classifier XML file (default: 'pre-trained models/haarcascade_frontalface_default.xml').
    - scale_factor: float. Specifies how much the image size is reduced at each image scale (default: 1.1).
    - min_neighbors: int. Specifies the number of neighbors a region needs to have to be considered as a face (default: 3).
    - min_size: tuple of ints. Specifies the minimum size for a detected region to be considered as a face (default: (30, 30)).
    - blur_kernel_size: tuple of ints. Specifies the kernel size for Gaussian blur (default: (25, 25)).
    - blur_level: int. Specifies the level of blur to be applied to the face (default: 100).
    - return_bb: boolean. if true, return the bounding box cordinate, if false nothing

    Returns: by default
    None.

    Raises:
    - IOError: If the input image cannot be loaded or the face cascade classifier XML file cannot be loaded.

    Examples:
    - blur_face('test_img.jpg', 'blurred_img.jpg'): blurs faces in 'test_img.jpg' and saves the output as 'blurred_img.jpg'.
    pre-trained models\haarcascade_frontalface_default.xml
    """


    # Load the input image
    img = cv2.imread(img_path)

    # Check if the image loaded correctly
    if type(img) == None:
        raise IOError("Unable toe load the image")


    # Load the face detection cascade
    face_cascade = cv2.CascadeClassifier(cascade_classifier)

    # Check if the cascade classifier is loaded correctly
    if face_cascade.empty():
        raise IOError('Unable to load the face cascade classifier xml file')


    # Convert the input image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray_img, scaleFactor=scale_factor, minNeighbors=min_neighbors, minSize=min_size)

    # Loop over all the faces and blur them
    for (x, y, w, h) in faces:

        # Create a region of interest (ROI) around the face
        region = img[y:y+h, x:x+w]

        # Apply Gaussian blur to the ROI
        blurred_region = cv2.GaussianBlur(region, blur_kernel_size, blur_level)

        # Replace the original ROI with the blurred ROI
        img[y:y+h, x:x+w] = blurred_region


    # Save the output image
    cv2.imwrite(blured_img_name, img)

    # if returning bounding box 
    if return_bb:
        return x, y, w + x, h + y


blur_face(".\\faces\\faces\\face7.jpg")

img = cv2.imread('cascade_output.jpg')
cv2.imshow("Cascade Classifier", img)
cv2.waitKey()
cv2.destroyAllWindows()

## Using `YOLO v8`

In [57]:
def yolo_face_blur(img_path, out_img_name = "Yolo_output.jpg",weights_path=".\\pretrained_models\YOLO_Model.pt",
                   blur_kernel_size=(25,25), blur_level=100, return_bb = False ):
    
    """
    Applies blurring to detected faces in an image using a YOLO face detection model.

    Args:
        img_path (str): Path to the input image to be processed.
        out_img_name (str, optional): Name of the output image file to be saved after processing. Defaults to "model2.jpg".
        weights_path (str, optional): Path to the pre-trained YOLO model weights. Defaults to ".\\pretrained_models\YOLO_Model.pt".
        blur_kernel_size (tuple, optional): Kernel size for Gaussian blur. Defaults to (25, 25).
        blur_level (int, optional): Blur level for Gaussian blur. Defaults to 100.
        return_bb: boolean. if true, return the bounding box cordinate, if false nothing
        
    Returns: by defualt
        None.

    """

    # Build the model
    model = YOLO('yolov8n.pt')
    model = YOLO(weights_path)

    # start detecting
    results = model(img_path)
    img = cv2.imread(img_path)

    # loop over detections
    for result in results:
        
        # get the bounding boxes results
        boxes = result.boxes

        if boxes:

            # loop over each box for each detect
            for box in boxes:

                # assigning the cordinate for the bounding box
                bouding_cordinate = box.xyxy
                x1, y1, x2, y2 = ( int(bouding_cordinate[0,0].item()), int(bouding_cordinate[0,1].item()), 
                                   int(bouding_cordinate[0,2].item()), int(bouding_cordinate[0,3].item()) )

                # delete all the negative cordinates
                x1, y1, x2, y2 = max(x1,0), max(y1, 0), max(x2, 0), max(y2, 0)

                # select the region & apply Gaussian Blur & put it on the original image
                region = img[y1:y2, x1:x2]
                blurred_region = cv2.GaussianBlur(region, blur_kernel_size, blur_level)
                img[y1:y2, x1:x2] = blurred_region


    # Save the image
    cv2.imwrite(out_img_name, img)
    
    # if returning bounding box 
    if return_bb:
        return x1, y1, x2, y2

yolo_face_blur(".\\faces\\faces\\face3.jpg")

img = cv2.imread('Yolo_output.jpg')
cv2.imshow("Yolov8", img)
cv2.waitKey()
cv2.destroyAllWindows()

Ultralytics YOLOv8.0.39  Python-3.9.13 torch-1.13.1+cpu CPU
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 D:\Projects\ProgressSoft_Assignment\Problem1\faces\faces\face3.jpg: 448x640 11 faces, 94.9ms
Speed: 1.0ms pre-process, 94.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


## Evaluation

#### Between the two models

In [58]:
def calculate_iou(box1, box2):
    """
    Calculate intersection over union (IoU) between two bounding boxes
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - intersection
    iou = intersection / union
    return iou

img_path = ".\\faces\\faces\\face2.jpg"
box1 = blur_face(img_path, return_bb=True)
box2 = yolo_face_blur(img_path, return_bb=True)
print(box1)
print(box2)
print("The IOU between the models is: ", calculate_iou(box1, box2))

Ultralytics YOLOv8.0.39  Python-3.9.13 torch-1.13.1+cpu CPU
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 D:\Projects\ProgressSoft_Assignment\Problem1\faces\faces\face2.jpg: 480x640 1 face, 92.8ms
Speed: 1.0ms pre-process, 92.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


(152, 73, 309, 230)
(144, 56, 322, 262)
The IOU between the models is:  0.6722210101450856


In [59]:
def yolo_box(img_path, weights_path=".\\pretrained_models\YOLO_Model.pt"):
    """
    funcion returns the detecting bounding boxes for the yolo model

    Args:
        img_path (str): the path for the image
        weights_path (str, optional): the path for the weights. Defaults to ".\pretrained_models\YOLO_Model.pt".

    Returns:
        tuple: detection bouding boxes
    """

    model = YOLO('yolov8n.pt')
    model = YOLO(weights_path)

    results = model(img_path)
    img = cv2.imread(img_path)

    the_box = []
    for result in results:
        boxes = result.boxes
        if boxes:
            for box in boxes:
                bouding_cordinate = box.xyxy
                
                x1, y1, x2, y2 = ( int(bouding_cordinate[0,0].item()), int(bouding_cordinate[0,1].item()), 
                                   int(bouding_cordinate[0,2].item()), int(bouding_cordinate[0,3].item()) )

                x1, y1, x2, y2 = max(x1,0), max(y1, 0), max(x2, 0), max(y2, 0)
                the_box.append([0, x1, y1, x2, y2]) 

    return the_box

In [ ]:
def cascade_box(img_path, cascade_classifier = ".\pretrained_models\haarcascade_frontalface_default.xml"):
    """
    function return the detecting bounding box for the haar cascade pre-trained model

    Args:
        img_path (str): path for the image
        cascade_classifier (str, optional): path for the pretrained model. Defaults to ".\pretrained_models\haarcascade_frontalface_default.xml".

    Returns:
        tuple: detection bounding boxes
    """

    img = cv2.imread(img_path)
    face_cascade = cv2.CascadeClassifier(cascade_classifier)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=3, minSize=(30,30))
    out_box = []

    for (x, y, w, h) in faces:
        
        out_box.append([0,x,y,w,h])

    return out_box

In [61]:
def compute_iou_matrix(boxes_true, boxes_pred):
    """
    Computes the IoU matrix between the true and predicted bounding boxes.
    :param boxes_true: numpy array of shape (n_true_boxes, 5) where each row is (label, x_min, y_min, x_max, y_max)
    :param boxes_pred: numpy array of shape (n_pred_boxes, 5) where each row is (label, x_min, y_min, x_max, y_max)
    :return: numpy array of shape (n_true_boxes, n_pred_boxes) representing the IoU matrix
    """
    # Compute the intersection areas
    x_min = np.maximum(boxes_true[:, 1].reshape(-1, 1), boxes_pred[:, 1].reshape(1, -1))
    y_min = np.maximum(boxes_true[:, 2].reshape(-1, 1), boxes_pred[:, 2].reshape(1, -1))
    x_max = np.minimum(boxes_true[:, 3].reshape(-1, 1), boxes_pred[:, 3].reshape(1, -1))
    y_max = np.minimum(boxes_true[:, 4].reshape(-1, 1), boxes_pred[:, 4].reshape(1, -1))
    intersection_area = np.maximum(x_max - x_min, 0) * np.maximum(y_max - y_min, 0)

    # Compute the union areas
    area_true = (boxes_true[:, 3] - boxes_true[:, 1]) * (boxes_true[:, 4] - boxes_true[:, 2])
    area_pred = (boxes_pred[:, 3] - boxes_pred[:, 1]) * (boxes_pred[:, 4] - boxes_pred[:, 2])
    union_area = area_true.reshape(-1, 1) + area_pred.reshape(1, -1) - intersection_area

    # Compute the IoU matrix
    iou_matrix = intersection_area / union_area

    return iou_matrix

In [62]:
def evaluate_model(ground_truth_path, bb_model_func, iou_threshold):
    
    # Get list of image paths
    image_paths = [os.path.join(ground_truth_path, 'images', filename) for filename in os.listdir(os.path.join(ground_truth_path, 'images')) if filename.endswith('.jpg')]

    # Loop through images and compute metrics
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    total_iou = 0
    total_ground_truth_boxes = 0

    for image_path in image_paths[:100]:
        
        # Load image and ground truth boxes
        image = cv2.imread(image_path)
        label_path = os.path.join(ground_truth_path, 'labels', os.path.splitext(os.path.basename(image_path))[0] + '.txt')
        with open(label_path, 'r') as f:
            ground_truth_boxes = np.array([list(map(float, line.strip().split())) for line in f])

        # Get predicted boxes and convert to numpy array
        predicted_boxes = bb_model_func(image_path)
        predicted_boxes = np.array(predicted_boxes) / image.shape[0]

        if ground_truth_boxes != []:

            iou_matrix = compute_iou_matrix(ground_truth_boxes, predicted_boxes)
            iou_matrix = np.where(iou_matrix >= iou_threshold, 1, 0)

            # Compute true positives, false positives, and false negatives
            true_positives += np.sum(np.sum(iou_matrix, axis=0) == 1)
            false_positives += np.sum(np.sum(iou_matrix, axis=1) == 0)
            false_negatives += np.sum(np.sum(iou_matrix, axis=0) == 0)
            total_iou += np.sum(np.max(iou_matrix, axis=0))
            total_ground_truth_boxes += len(ground_truth_boxes)

    # Compute precision, recall, and IoU
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    iou = total_iou / total_ground_truth_boxes

    return precision, recall, iou

In [63]:
precision, recall, iou = evaluate_model(".\\roboflow_data\\test", yolo_box,0.1)

Ultralytics YOLOv8.0.39  Python-3.9.13 torch-1.13.1+cpu CPU
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 D:\Projects\ProgressSoft_Assignment\Problem1\roboflow_data\test\images\-I1-MS09uaqsLdGTFkgnS0Rcg1mmPyAj95ySg_eckoM_jpeg_jpg.rf.ca3d216ca28be32265e365299a0af4b6.jpg: 640x640 3 faces, 107.0ms
Speed: 1.0ms pre-process, 107.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
Ultralytics YOLOv8.0.39  Python-3.9.13 torch-1.13.1+cpu CPU
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 D:\Projects\ProgressSoft_Assignment\Problem1\roboflow_data\test\images\0ad90195-cd77-489e-bf85-08c83b80d3e0_jpg.rf.3317b17670bf4318a3b12a9805730774.jpg: 640x640 6 faces, 114.9ms
Speed: 0.0ms pre-process, 114.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Ultralytics YOLOv8.0.39  Python-3.9.13 torch-1.13.1+cpu CPU
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GF

In [64]:
print(f"IoU threshold: {0.1}")
print(f"Precision: {(precision * 100 ).round(3)}%")
print(f"recall: {(recall* 100).round(3)}%")
print(f"IoU: {(iou * 100).round(3)}%")

IoU threshold: 0.1
Precision: 4.412%
recall: 4.027%
IoU: 4.511%
